In [5]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

In [6]:
# data locations
raw_data_path = '../data/peaks_data'
outbase = '../output/peaks_data'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase
motif_path = '%s/../motifs' % raw_data_path

# training hyperparameters
batch_size = 50
num_batches_in_train = int(41305 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [7]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

reload(fxns)
print(test_out_path)
(seqt, labelt), info = fxns.get_seq_and_label(test_out_path, num_epochs=None)
seqt_batch, labelt_batch = tf.train.shuffle_batch([seqt, labelt],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

inputs are of size 1200
outputs are of size 12
../output/peaks_data/texttest


In [8]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import motif_model; reload(motif_model)
filters = motif_model.get_motifs(motif_path)
modelname='peaks_motifconv'
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = motif_model.get_motif_model(info['seq_len'], info['label_len'], conv_infos, filters)

('max filter length set to', 21)
150.0
75.0
38.0
1200 h 4 38.0


In [9]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [10]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.6007275724411011
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.5003329145908356
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.4382620211442312
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.4014534839987755
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.3781718556880951
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.36220197260379794
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.35052662474768503
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.34174059838056564
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.33490647885534497
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.32939435839653014
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.32939435839653014
Epoch: 0 global_step 275 i 274 Avg loss in epoch(incomplete): 0.3248913269693201
Epoch: 0 global_step 3

In [11]:
#
# get test data

global_step_to_load_test = 1652

scores = []
labels = []

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=None)
    saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load_test))
    
    idx = 0
    while idx < 4348 / batch_size:
        batch_Xt, batch_Yt = sess.run([seqt_batch, labelt_batch])
        batch_logits = sess.run(logits, feed_dict={X: batch_Xt, Y: batch_Yt})
        scores.append(batch_logits)
        labels.append(batch_Yt)
        #print('asdf')
        idx += 1
        if idx % 100 == 0:
            print(idx)

scores_arr = np.concatenate(scores, axis=0)
labels_arr = np.concatenate(labels, axis=0)

INFO:tensorflow:Restoring parameters from log/peaks_motifconv-1652
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


In [12]:
accuracies = np.mean((scores_arr > 0).astype(int) == labels_arr.astype(int), axis=0)

print('accuracies:')
print(accuracies)

import sklearn.metrics
print('aucs:')
aucs = [sklearn.metrics.roc_auc_score(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(aucs)

import sklearn.metrics
def prec_recall(ys_true, ys_hat):
    ys, xs, thresholds = sklearn.metrics.precision_recall_curve(ys_true, ys_hat)
    return sklearn.metrics.auc(xs, ys, reorder=True)

print('auprcs:')
auprcs = [prec_recall(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(auprcs)

props = np.mean(labels_arr, axis=0)

import json
ids = sorted(json.loads(info['tf_to_pos'].replace("'", '"')).keys())

from IPython.display import display_pretty, display_html
import pandas as pd
results = pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs')
display_html(results.to_html(), raw=True)
results.to_csv('stats.' + modelname + '.tsv', index_label='id', sep='\t')

accuracies:
[0.89218391 0.90344828 0.89678161 0.90068966 0.91057471 0.91816092
 0.92114943 0.98       0.90689655 0.8908046  0.97471264 0.9045977 ]
aucs:
[0.4286222474699056, 0.6471407367018054, 0.4532416735129877, 0.6345856847787041, 0.4080845440993128, 0.4977188394951978, 0.5442822000275029, 0.35120294649766365, 0.6110907696881503, 0.3603925297113752, 0.42243782161234994, 0.5170710797446456]
auprcs:
[0.09172802958020329, 0.13438808018222148, 0.09101745554824742, 0.14345659407416966, 0.07419584422826453, 0.08871942196419141, 0.09713572415214303, 0.021039345944296694, 0.12342984539682292, 0.08542903803703411, 0.02484537088919657, 0.10089759343274307]


,acc,aucs,auprcs,props
NFKB2,0.980000,0.351203,0.021039,0.020000
SPI1,0.890805,0.360393,0.085429,0.109195
MAX,0.910575,0.408085,0.074196,0.089425
TAF1,0.974713,0.422438,0.024845,0.025287
CTCF,0.892184,0.428622,0.091728,0.107816
GATA2,0.896782,0.453242,0.091017,0.103218
MYC,0.918161,0.497719,0.088719,0.081839
TBP,0.904598,0.517071,0.100898,0.095402
NFKB1,0.921149,0.544282,0.097136,0.078851
PAX5,0.906897,0.611091,0.123430,0.093103
